# Hardware and Software Setup

## Chromatic Tuner

First we need a chromatic tuner that can tell us whether we are any close to our target frequency when we are tuning our piano. Here are some that I suggest for Linux:

### `kguitune`

http://www.oocities.org/harpin_floh/kguitune_page.html

Available as a binary as

```
sudo apt-get install gtkguitune
```

### `lingot`

http://www.nongnu.org/lingot/

Available as a binary as

```
sudo apt-get install lingot
```

## Recording Program

We'll try to do all our recording programmatically in Julia, but it may come handy to have an external software for troubleshooting when setting up the hardware. I recommend the following software in Linux:

#### `Kwave`

Available as a binary as

```
sudo apt-get install kwave
```

#### `Audacity`

Available as a binary as

```
sudo apt-get install audacity
```

## Julia Audio Processing

* Julia has a family of packages for audio processing here: https://github.com/JuliaAudio . 

* This package for handling WAV files may also come handy: https://github.com/dancasimiro/wav.jl

* Here are neat examples of audio processing: https://nbviewer.jupyter.org/github/JuliaAudio/JuliaAudioExamples/blob/master/MP3%20Encoding%20Loss.ipynb

In [4]:
using SampledSignals
using LibSndFile
using FileIO
using PlotlyJS

### Playing 

In [5]:
snd = load("audio/test00.wav")

301954-frame, 2-channel SampleBuf{FixedPointNumbers.Fixed{Int16,15}, 2}
6.84702947845805s sampled at 44100.0Hz
▄▄▄▄▃▃▃▄▅▄▄▄▄▃▇██▇▇▆▇▇▆▇▇▇▆▇▇▆▆▆▆▆▆▆▆▅▄▄▄▄▃▄▃▃▃▄▄▃▃▄▄▄▄▃▃▃▄▄▄▃▃▃▃▃▃▄▃▄▃▄▄▄▃▃▄▃▄▄
▄▄▄▄▃▃▃▄▅▄▄▄▄▃▇██▇▇▆▇▇▆▇▇▇▆▇▇▆▆▆▆▆▆▆▆▅▄▄▄▄▃▄▃▃▃▄▄▃▃▄▄▄▄▃▃▃▄▄▄▃▃▃▃▃▃▄▃▄▃▄▄▄▃▃▄▃▄▄

### Device Recording Through [`PortAudio.jl`](https://github.com/JuliaAudio/PortAudio.jl)

Somehow the master branch of `PortAudio.jl` was left behind in the Julia development. In order to install add the package, you'll need to indicate the v1.0-compatible branch manually:

```julia
(v1.0) pkg> add PortAudio#julia1
```

At first I was encountering the following error:
```julia
julia> microph = "default"
"default"

julia> stream = PortAudioStream(microph, microph)
julia: symbol lookup error: /home/edo/.julia/packages/PortAudio/VaI1c/src/../deps/usr/lib/pa_shim_x86_64-linux-gnu.so: undefined symbol: PaUtil_GetRingBufferWriteAvailable

```

I was able to solve it installing `RingBuffers` in the following sequence:
```julia
(v1.0) pkg> add RingBuffers#master
(v1.0) pkg> build RingBuffers
(v1.0) pkg> build PortAudio
```

In [1]:
using PortAudio, SampledSignals, LibSndFile, FileIO

ALSA lib pcm.c:2495:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2495:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2495:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:867:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:867:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:867:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:867:(find_matching_chmap) Found no matching channel map
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock


In [2]:
PortAudio.devices()

13-element Array{PortAudio.PortAudioDevice,1}:
 PortAudio.PortAudioDevice("HDA Intel PCH: ALC3227 Analog (hw:0,0)", "ALSA", 2, 2, 44100.0, 0)
 PortAudio.PortAudioDevice("HDA Intel PCH: HDMI 0 (hw:0,3)", "ALSA", 0, 8, 44100.0, 1)        
 PortAudio.PortAudioDevice("Yeti Nano: USB Audio (hw:1,0)", "ALSA", 2, 2, 44100.0, 2)         
 PortAudio.PortAudioDevice("sysdefault", "ALSA", 128, 128, 48000.0, 3)                        
 PortAudio.PortAudioDevice("front", "ALSA", 0, 2, 44100.0, 4)                                 
 PortAudio.PortAudioDevice("surround40", "ALSA", 0, 2, 44100.0, 5)                            
 PortAudio.PortAudioDevice("surround51", "ALSA", 0, 2, 44100.0, 6)                            
 PortAudio.PortAudioDevice("surround71", "ALSA", 0, 2, 44100.0, 7)                            
 PortAudio.PortAudioDevice("hdmi", "ALSA", 0, 8, 44100.0, 8)                                  
 PortAudio.PortAudioDevice("pulse", "ALSA", 32, 32, 44100.0, 9)                               
 Po

In [3]:
# Grab the Yeti Nano as the input device (microphone)
microph = PortAudio.devices()[3]

PortAudio.PortAudioDevice("Yeti Nano: USB Audio (hw:1,0)", "ALSA", 2, 2, 44100.0, 2)

In [3]:
? PortAudioStream

search: PortAudioStream



```
PortAudioStream(inchannels=2, outchannels=2; options...)
PortAudioStream(duplexdevice, inchannels=2, outchannels=2; options...)
PortAudioStream(indevice, outdevice, inchannels=2, outchannels=2; options...)
```

Audio devices can either be `PortAudioDevice` instances as returned by `PortAudio.devices()`, or strings with the device name as reported by the operating system. If a single `duplexdevice` is given it will be used for both input and output. If no devices are given the system default devices will be used.

Options:

  * `eltype`:       Sample type of the audio stream (defaults to Float32)
  * `samplerate`:   Sample rate (defaults to device sample rate)
  * `blocksize`:    Size of the blocks that are written to and read from the audio                 device. (Defaults to 4096)
  * `synced`:       Determines whether the input and output streams are kept in                 sync. If `true`, you must read and write an equal number of                 frames, and the round-trip latency is guaranteed constant. If                 `false`, you are free to read and write separately, but                 overflow or underflow can affect the round-trip latency.
  * `warn_xruns`:   Display a warning if there is a stream overrun or underrun,                 which often happens when Julia is compiling, or with a                 particularly large GC run. This can be quite verbose so is                 false by default.


In [4]:
# Open the microphone stream
stream = PortAudioStream(microph, 2, 0)

PortAudioStream{Float32}
  Samplerate: 44100.0Hz
  Buffer Size: 4096 frames
  2 channel source: "Yeti Nano: USB Audio (hw:1,0)"

In [17]:
propertynames(stream)

(:samplerate, :blocksize, :stream, :warn_xruns, :sink, :source, :errbuf, :errtask, :bufinfo)

In [18]:
stream.samplerate

44100.0

In [29]:
s = 3
samples = ceil(Int, s*stream.samplerate)

# Record 10 seconds of the stream
buf = read(stream, samples)

132300-frame, 2-channel SampleBuf{Float32, 2}
3.0s sampled at 44100.0Hz
▂▂▂▂▂▄▃▂▂▃▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▁▂▃▅▅▅▅▅▇▆▅▅▅▅▅▄▄▅▅▅▅▆▅▅▅▄▅▄▅▄▃▃▄▄▄▄▄▄▃▃▃▄▅▂▂▂▂▂▂▂▅█▄▄▃
▂▂▂▂▂▄▃▂▂▃▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▁▂▃▅▅▅▅▅▇▆▅▅▅▅▅▄▄▅▅▅▅▆▅▅▅▄▅▄▅▄▃▃▄▄▄▄▄▄▃▃▃▄▅▂▂▂▂▂▂▂▅█▄▄▃

In [36]:
save("audio/test01.wav", buf)

In [37]:
snd = load("audio/test01.wav")

132300-frame, 2-channel SampleBuf{Float32, 2}
3.0s sampled at 44100.0Hz
▂▂▂▂▂▄▃▂▂▃▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▁▂▃▅▅▅▅▅▇▆▅▅▅▅▅▄▄▅▅▅▅▆▅▅▅▄▅▄▅▄▃▃▄▄▄▄▄▄▃▃▃▄▅▂▂▂▂▂▂▂▅█▄▄▃
▂▂▂▂▂▄▃▂▂▃▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▁▂▃▅▅▅▅▅▇▆▅▅▅▅▅▄▄▅▅▅▅▆▅▅▅▄▅▄▅▄▃▃▄▄▄▄▄▄▃▃▃▄▅▂▂▂▂▂▂▂▅█▄▄▃